# Isolated Bacterial Genome Analysis

This part contains code to analyse the isolated bacterial genome. It contains the following steps:

## Split chromesome and plasmids based on `genomad` classification



In [1]:
from Bio import SeqIO
import pandas as pd
import re
from Bio import SeqIO
from dataclasses import replace
import os
from BCBio import GFF
from Bio.Seq import Seq

rcs = list(SeqIO.parse("D45.gbff", "genbank"))

plasmid_ids = ["D45_12", "D45_16", "D45_20", "D45_25", "D45_28", "D45_29", "D45_30", "D45_34", "D45_36"]
rcs_chr = [x for x in rcs if x.id not in plasmid_ids]
rcs_plasmids = [x for x in rcs if x.id in plasmid_ids]

# write chrome and plasmids to separate files
SeqIO.write(rcs_plasmids, "D45_plasmids.gbk", "genbank")
SeqIO.write(rcs_chr, "D45_chr.gbk", "genbank")

FileNotFoundError: [Errno 2] No such file or directory: 'D45.gbff'

# Merge bacteria protein annotations

### Extract KOs from EggNOG annotation

In [ ]:
import pandas as pd
import os

df = pd.read_csv('p0062_anno_eggnog.tsv', sep='\t')
tbl = df.iloc[:,[0,8]]
tbl = tbl[tbl.kegg_ko.notna()].copy()
tbl['ctgid'] = tbl.apply(lambda x:'_'.join(x['prot_id'].split('_')[:-1]), axis=1)

fs = tbl[["ctgid", "kegg_ko"]].values.tolist()

rst = []
for item in fs:
    kos = item[1]
    ks = kos.replace('ko:', '')
    for ko in ks.split(','):
        rst.append([item[0], ko])

df2 = pd.DataFrame(rst)
df2.to_csv('kos.txt', sep='\t', header=False, index=False)

### Annotate protein and add the annotation to the GBK file

##### GBK from PhageGAN or Pharokka pipeline

```bash
# PhageGAN pipeline
nextflow ...

# or Pharokka pipeline
sbatch ... anno_contig.slurm ...
```

##### MMseqs2 NR annotation

```bash
# TODO: annotate using MMseqs2 NR -> rstAnnoNR.m8

# Pretty NR annotation
pretty_mmseqs_nr_annotation.py -i rstAnnoNR.m8 -o rstAnnoNR.tsv

# Add annotation to the GBK file
add_annotation2gbk.py -i rstAnnoNR.tsv -g old.gbk -a 'function' -A 'anno_nr' -o new.gbk

# example
for f in $(ls gbk_pipeline/*.gbk);do
    fid=$(basename $f | sed 's/.gbk//g')
    add_annotation2gbk.py -i rstAnnoNR.tsv -g $f -a 'function' -A 'anno_nr' -o gbk_nr/${fid}.gbk
done
```

##### EggNOG-mapper annotation

```bash
# TODO: annotate using emapper

# Pretty emapper annotation
parse_eggnog.py -i output_emapper.emapper.annotations -o em.tsv

# Add annotation to the GBK file
add_annotation2gbk.py -i em.tsv -g D45mix.gbk -a 'Preferred_name' -A 'anno_EggNOG_name' -o D45mix_EMname.gbk
add_annotation2gbk.py -i em.tsv -g D45mix_EMname.gbk -a 'Description' -A 'anno_EggNOG_desc' -o D45mix_EMname_desc.gbk
add_annotation2gbk.py -i em.tsv -g D45mix_EMname_desc.gbk -a 'PFAMs' -A 'anno_EggNOG_PFAM' -o D45mix_EMname_desc_PFAM.gbk
add_annotation2gbk.py -i em.tsv -g D45mix_EMname_desc_PFAM.genbank -a 'max_annot_lvl' -A 'anno_EggNOG_lvl' -o D45mix_EMname_desc_PFAM_lvl.gbk
```

#### 01-phagegan28samples

`workflow_exp/data/00-raw/01-phagegan28samples/analysis`

In [ ]:
fin = "final_gbk/D45mix_em_lvl.genbank"
rcs = list(SeqIO.parse(fin, "genbank"))

for rc in rcs:
    for feature in rc.features:
        product = feature.qualifiers.get('product')
        if product and "hypothetical protein" in product:
            anno_keys = [k for k in feature.qualifiers.keys() if 'anno_' in k]
            if len(anno_keys) > 0:
                anno = [feature.qualifiers.get(k)[0] for k in anno_keys]
                anno = [x for x in anno if x != 'hypothetical protein']
                if len(anno) > 0:
                    feature.qualifiers.update({'product': ';'.join(anno)})

with open(fout, 'w') as fh:
    SeqIO.write(rcs, fh, "genbank")

#### 91407/124278 (73.4%) of the phage proteins have a hit in the nr database

Input: `rstBhNR.m8`

1. Extract NR functions from .m8 file,
  ```bash
  pretty_mmseqs_nr_annotation.py -i rstAnnoNR.m8 -o rstAnnoNR.tsv
  ```

2. Convert NR annotation to a dictionary `anno`,

  ```json
  {'D45s01_15_3899': 
    {'target': 'WP_063972546.1',
    'pident': 64.4,
    'bits': 146,
    'evalue': 3.451e-37,
    'taxid': 37914,
    'function2': nan,
    'function_opt': nan},

  'D45s01_16_3995': 
    {'target': 'WP_107757771.1',
    'pident': 96.9,
    'bits': 953,
    'evalue': 5.766e-310,
    'taxid': 2617939,
    'function2': 'LLM class flavin-dependent oxidoreductase',
    'function_opt': '5,10-methylene tetrahydromethanopterin reductase;;putative monooxygenase'},
  ...
  }
```

#### 28 个 samples 总共有 1297 个 contigs 的 124278 个 proteins header 转换为新的 feature_id

Input: `phage28.faa`

| prot_id    |   loc_range | genome_id   | feature_id         |
|:-----------|------------:|:------------|:-------------------|
| D45mix_1_0 |       1_267 | D45mix_1    | D45mix_1_1_267     |
| D45mix_1_1 |     306_791 | D45mix_1    | D45mix_1_306_791   |
| D45mix_1_2 |    788_1039 | D45mix_1    | D45mix_1_788_1039  |

In [ ]:
def faa_header_to_df(fin):
    # ID mapping of FASTA
    rcs_fasta = list(SeqIO.parse(fin, "fasta"))
    df = pd.DataFrame([x.description.split(' ') for x in rcs_fasta], columns=['prot_id', 'loc_range'])
    df['genome_id'] = df.apply(lambda x: '_'.join(x['prot_id'].split('_')[:-1]), axis=1)
    df['feature_id'] = df.apply(lambda x: '_'.join([x['genome_id'], x['loc_range']]), axis=1)

    return df

fin = "phage28.faa"
id_mapping_faa = faa_header_to_df(fin)
id_mapping_faa.head()

```bash
# GFF -- FASTA_FAA --> GBK
gff2gbk.py -f D45mix.gff -a D45mix_scaffolds.fasta -p phanotate_aas.fasta -o D45mix_manual.gbk

# anno_nr >> GBK
add_annotation2gbk.py -i rstAnnoNR.tsv -g D45mix_manual.gbk -a 'function' -A 'anno_nr' -o final.gbk
```